In [42]:
import numpy as np
import pandas as pd
import math
from typing import Tuple

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [43]:
def EvenDifference(points):
    """
    Calculates the finite difference table for evenly spaced nodes.
    
    Args:
        points (list of tuples): A list of (x, y) data points, sorted by x.

    Returns:
        pandas.DataFrame: The full difference table.
    """
    x_values = [p[0] for p in points]
    y_values = [p[1] for p in points]
    n = len(y_values)

    # Internal calculation table
    diff_calc_table = np.full((n, n), np.nan)
    diff_calc_table[:, 0] = y_values
    for j in range(1, n):
        for i in range(n - j):
            diff_calc_table[i, j] = diff_calc_table[i+1, j-1] - diff_calc_table[i, j-1]

    # Format the output DataFrame
    data = {
        'x_i': x_values,
        'y_i': y_values
    }
    for j in range(1, n):
        # Pad with NaN to maintain table shape
        col_name = f'Order {j}'
        col_data = np.full(n, np.nan)
        col_data[:(n-j)] = diff_calc_table[:(n-j), j]
        data[col_name] = col_data
    
    df = pd.DataFrame(data)
    return df


# Stirling


## Algorithm

0. Conditions
* Used for **evenly spaced nodes**: $x_k = x_0 + kh$, $k \in \mathbb{Z}$.
* Requires an **odd number of nodes** (which means an even degree $n$).
* The formula originates from the **central node** $x_0$.
* It is the average of the Gauss I and Gauss II formulas.

1. Finite Difference Table

Given $n+1$ nodes (where $n$ is even), with $x_0$ as the central node.
A standard finite difference table is computed.

*(This is performed by the `EvenDifference(points)` function, which can be found in `Week6-Interpolation/pp2_Newton/pp2b_NewtonFixedGap.ipynb`)*

2. Coefficient Selection

The coefficients $C_i$ (from the slide's formula) are selected from the difference table symmetrically around $y_0$:
* $C_0 = y_0$
* $C_1 = \frac{\Delta y_{-1} + \Delta y_0}{2}$
* $C_2 = \Delta^2 y_{-1}$
* $C_3 = \frac{\Delta^3 y_{-2} + \Delta^3 y_{-1}}{2}$
* $C_4 = \Delta^4 y_{-2}$
* ...
* **General form:**
    * $C_{2k} = \Delta^{2k} y_{-k}$
    * $C_{2k-1} = \frac{\Delta^{2k-1} y_{-k} + \Delta^{2k-1} y_{-(k-1)}}{2}$

3. Polynomial Construction in $t$

The polynomial is constructed in the variable $t = \frac{x - x_0}{h}$.
The formula from the slide is:
$P(t) = C_0 + C_1 \frac{t}{1!} + C_2 \frac{t^2}{2!} + C_3 \frac{t(t^2-1^2)}{3!} + C_4 \frac{t^2(t^2-1^2)}{4!} + C_5 \frac{t(t^2-1^2)(t^2-2^2)}{5!} + \dots$

This can be written as $P(t) = \sum_{i=0}^{n} D_i B_i(t)$

1.  **Calculate Main Coefficients $D_i$:**
    $D_i = \frac{C_i}{i!}$

2.  **Calculate Basis Polynomials $B_i(t)$:**
    * $B_0(t) = 1$
    * $B_1(t) = t$
    * $B_2(t) = t^2$
    * $B_3(t) = t(t^2 - 1^2)$
    * $B_4(t) = t^2(t^2 - 1^2)$
    * $B_5(t) = t(t^2 - 1^2)(t^2 - 2^2)$
    * ...
    * **Recursive form:**
        * $B_{2k}(t) = B_{2k-1}(t) \cdot t$
        * $B_{2k+1}(t) = B_{2k-1}(t) \cdot (t^2 - k^2)$

3.  **Compute Total Polynomial $P(t)$:**
    $P(t) = \sum_{i=0}^{n} N_i(t) = \sum_{i=0}^{n} D_i B_i(t)$
    * The coefficients of each $N_i(t)$ are summed by degree to find the final coefficients of $P(t) = a_0 + a_1 t + \dots + a_n t^n$.

4.  **Output:** Return the intermediate steps table and the final coefficient list $a_k$ for $P(t)$.

In [44]:
def stirling_interpolation(points, x0_index):
    """
    Constructs the Stirling interpolation polynomial in the variable t = (x - x0) / h.
    
    Args:
        points (list of tuples): List of (x, y) data points. Must be evenly spaced
                                 and have an odd number of points.
        x0_index (int): The index of the central node (x0).

    Returns:
        step_pd (pd.DataFrame): DataFrame showing the intermediate steps.
        coeff_pd (pd.DataFrame): DataFrame of the final polynomial coefficients for P(t).
    """
    
    n = len(points) - 1
    if (n + 1) % 2 == 0:
        raise ValueError("Stirling's formula requires an odd number of nodes (even degree n).")
    
    # --- 1. Generate Difference Table ---
    diff_table_df = EvenDifference(points)

    # --- 2. Select Coefficients (Stirling Path) ---
    C_coeffs = []
    for i in range(n + 1):
        order_col = f'Order {i}' if i > 0 else 'y_i'
        
        if i == 0:
            # C_0 = y_0
            C_coeffs.append(diff_table_df[order_col].iloc[x0_index])
        elif i % 2 == 1: # Odd order: C_{2k-1} = (d_{2k-1} y_{-k} + d_{2k-1} y_{-(k-1)}) / 2
            k = (i + 1) // 2
            c1 = diff_table_df[order_col].iloc[x0_index - k]
            c2 = diff_table_df[order_col].iloc[x0_index - k + 1]
            C_coeffs.append((c1 + c2) / 2.0)
        else: # Even order: C_{2k} = d_{2k} y_{-k}
            k = i // 2
            C_coeffs.append(diff_table_df[order_col].iloc[x0_index - k])

    # --- 3. Build Polynomial P(t) ---
    steps_data = []
    N_coeffs_total = np.zeros(n + 1, dtype=float)
    
    # B_i(t) basis polynomials
    B_coeffs_list = []
    
    # B_0(t) = 1
    B_0 = np.array([1.0])
    B_coeffs_list.append(B_0)
    
    # B_1(t) = t
    B_1 = np.array([0.0, 1.0]) # [const, t]
    B_coeffs_list.append(B_1)

    # *** CORRECTED LOGIC FOR B_i(t) ***
    for i in range(2, n + 1):
        if i % 2 == 0: # Even: B_{2k} = t * B_{2k-1}
            # B_{2k-1} is the previous basis poly, at index i-1
            B_i = np.convolve(B_coeffs_list[i-1], [0, 1])
        else: # Odd: B_{2k+1} = (t^2 - k^2) * B_{2k-1}
            k = (i - 1) // 2
            # B_{2k-1} is the basis poly at index i-2
            B_i = np.convolve(B_coeffs_list[i-2], [-k**2, 0, 1.0])
        
        B_coeffs_list.append(B_i)

    # Calculate final polynomial
    for i in range(n + 1):
        D_i = C_coeffs[i] / math.factorial(i)
        B_i = B_coeffs_list[i]
        
        # N_i(t) = D_i * B_i(t)
        Ni_coeffs = D_i * B_i
        
        # Add to total polynomial (pad with zeros)
        N_coeffs_total[:len(Ni_coeffs)] += Ni_coeffs

        # Store intermediate steps for printing
        steps_data.append({
            'i': i,
            'Difference Coeff (C_i)': C_coeffs[i],
            'D_i = C_i / i!': D_i,
            'B_i(t) Coeffs (low->high)': B_i.tolist(),
            'N_i(t) Coeffs (low->high)': Ni_coeffs.tolist()
        })

    step_pd = pd.DataFrame(steps_data)
    coeff_pd = pd.DataFrame({
        'Degree (t)': np.arange(n + 1),
        'Coeff (low->high)': N_coeffs_total
    })

    return step_pd, coeff_pd

## Result

In [45]:
# 1. Define the data points from the image
points = [
    (1.4, 2.4347052),
    (1.5, 2.5473627),
    (1.6, 2.6495552),
    (1.7, 2.7412610),
    (1.8, 2.8225627),
    (1.9, 2.8936474),
    (2.0, 2.9548000),
    (2.1, 3.0064203),
    (2.2, 3.0489801)
]

# 2. Set the central node index
# 9 points (indices 0-8), the center is index 4
x0_index = 4
x0_val = points[x0_index][0]
h = points[1][0] - points[0][0]

In [46]:
print("--- Generated Finite Difference Table ---")
df = EvenDifference(points)

df.style

--- Generated Finite Difference Table ---


,x_i,y_i,Order 1,Order 2,Order 3,Order 4,Order 5,Order 6,Order 7,Order 8
0,1.400000,2.434705,0.112658,-0.010465,-0.000022,0.000104,0.000000,-0.000007,0.000031,-0.000114
1,1.500000,2.547363,0.102193,-0.010487,0.000083,0.000104,-0.000007,0.000024,-0.000084,nan
2,1.600000,2.649555,0.091706,-0.010404,0.000187,0.000098,0.000017,-0.000060,nan,nan
3,1.700000,2.741261,0.081302,-0.010217,0.000285,0.000115,-0.000043,nan,nan,nan
4,1.800000,2.822563,0.071085,-0.009932,0.000400,0.000072,nan,nan,nan,nan
5,1.900000,2.893647,0.061153,-0.009532,0.000472,nan,nan,nan,nan,nan
6,2.000000,2.954800,0.051620,-0.009060,nan,nan,nan,nan,nan,nan
7,2.100000,3.006420,0.042560,nan,nan,nan,nan,nan,nan,nan
8,2.200000,3.048980,nan,nan,nan,nan,nan,nan,nan,nan


In [47]:
# 3. Calculate the Gauss II polynomial
step_df, final_coeff_df = stirling_interpolation(points, x0_index)

print("\n--- Polynomial Construction Steps (Sterling) ---")
step_df.style


--- Polynomial Construction Steps (Sterling) ---


,i,Difference Coeff (C_i),D_i = C_i / i!,B_i(t) Coeffs (low->high),N_i(t) Coeffs (low->high)
0,0,2.822563,2.822563,[1.0],[2.8225627]
1,1,0.076193,0.076193,"[0.0, 1.0]","[0.0, 0.07619319999999985]"
2,2,-0.010217,-0.005109,"[0.0, 0.0, 1.0]","[-0.0, -0.0, -0.005108500000000182]"
3,3,0.000236,0.000039,"[0.0, -1.0, 0.0, 1.0]","[0.0, -3.9333333333391174e-05, 0.0, 3.9333333333391174e-05]"
4,4,0.000098,0.000004,"[0.0, 0.0, -1.0, 0.0, 1.0]","[0.0, 0.0, -4.07500000004779e-06, 0.0, 4.07500000004779e-06]"
5,5,0.000005,0.000000,"[0.0, 4.0, 0.0, -5.0, 0.0, 1.0]","[0.0, 1.733333333013102e-07, 0.0, -2.1666666662663775e-07, 0.0, 4.333333332532755e-08]"
6,6,0.000024,0.000000,"[0.0, 0.0, 4.0, 0.0, -5.0, 0.0, 1.0]","[0.0, 0.0, 1.3222222219826878e-07, 0.0, -1.6527777774783596e-07, 0.0, 3.3055555549567194e-08]"
7,7,-0.000027,-0.000000,"[0.0, -36.0, 0.0, 49.0, 0.0, -14.0, 0.0, 1.0]","[-0.0, 1.8964285712114962e-07, -0.0, -2.5812499997045365e-07, -0.0, 7.374999999155819e-08, -0.0, -5.267857142254156e-09]"
8,8,-0.000114,-0.000000,"[0.0, 0.0, -36.0, 0.0, 49.0, 0.0, -14.0, 0.0, 1.0]","[-0.0, -0.0, 1.0223214284209105e-07, -0.0, -1.3914930553506838e-07, -0.0, 3.9756944438590965e-08, -0.0, -2.8397817456136403e-09]"


## Further Testing

In [48]:
#Horner Test
def synthetic_division(a, c):
    """
    Perform synthetic division for polynomial p(x) with coefficients a,
    evaluated at x = c.

    Parameters:
        a (list[float]): coefficients of p(x) from highest to lowest degree
        c (float): the value to evaluate p(c)

    Returns:
        df (pd.DataFrame): table with columns [i, a_i, b_i*c, b_i]
        p_c (float): value of p(c)
        q_coeff (list[float]): coefficients of q(x) = (p(x) - p(c)) / (x - c)
    """

    n = len(a) - 1
    b = [0.0] * (n + 1)
    bc_values = [""] * (n + 1)

    b[n] = a[n]
    for i in range(n - 1, -1, -1):
        b[i] = a[i] + c * b[i + 1]
        bc_values[i + 1] = b[i + 1] * c

    # Prepare table (i from n to 0)
    df = pd.DataFrame({
        "i": list(range(n, -1, -1)),
        "a_i": [a[i] for i in range(n, -1, -1)],
        "b_i*c": [bc_values[i] for i in range(n, -1, -1)],
        "b_i = a_i + b_(i+1)*c": [b[i] for i in range(n, -1, -1)]
    })

    p_c = b[0]
    q_coeff = b[1:]
    return df, p_c, q_coeff, b

def all_derivatives(a, c):
    """
    Compute all derivatives p^(i)(c) using repeated Horner division
    and display in transposed table format.
    """
    coeffs = a.copy()
    degree = len(a) - 1
    results = []
    b0_list = []
    derivative_list = []

    # Perform repeated synthetic division
    for i in range(degree + 1):
        df, b0, next_coeff, b_all = synthetic_division(coeffs, c)
        results.append(b_all)
        b0_list.append(b0)
        derivative_list.append(b0 * math.factorial(i))
        coeffs = next_coeff
        if len(coeffs) == 0:
            break

    # Pad b_i lists for equal column length
    max_len = max(len(b) for b in results)
    for b in results:
        b.extend([None] * (max_len - len(b)))

    # Create DataFrame horizontally
    df = pd.DataFrame(results).T
    df.columns = [f"i={i}" for i in range(len(results))]

    # Insert first column for original a coefficients
    a_col = a + [None] * (df.shape[0] - len(a))
    df.insert(0, "a_i", a_col)

    # Add b_0 and p^(i)(c) rows
    df.loc["b_0"] = [None] + b0_list
    df.loc["p^(i)(c)"] = [None] + derivative_list

    # Add a row on top showing the value of c
    df.loc["c"] = [c] + [None] * (df.shape[1] - 1)
    df = df.loc[["c"] + [idx for idx in df.index if idx != "c"]]  # Move row to top

    return df

In [49]:
coeff_list = final_coeff_df['Coeff (low->high)'].tolist()

x_val = 1.63
t_val = (x_val - 1.6) / h
df2 = all_derivatives(coeff_list, t_val)
df2.style

,a_i,i=0,i=1,i=2,i=3,i=4,i=5,i=6,i=7,i=8
c,0.300000,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,2.822563,2.844950,0.073098,-0.005075,0.000044,0.000004,0.000000,0.000000,-0.000000,-0.000000
1,0.076154,0.074624,-0.005088,0.000042,0.000004,0.000000,0.000000,-0.000000,-0.000000,nan
2,-0.005112,-0.005100,0.000041,0.000004,0.000000,0.000000,-0.000000,-0.000000,nan,nan
3,0.000039,0.000040,0.000004,0.000000,0.000000,-0.000000,-0.000000,nan,nan,nan
4,0.000004,0.000004,0.000000,0.000000,-0.000000,-0.000000,nan,nan,nan,nan
5,0.000000,0.000000,0.000000,-0.000000,-0.000000,nan,nan,nan,nan,nan
6,0.000000,0.000000,-0.000000,-0.000000,nan,nan,nan,nan,nan,nan
7,-0.000000,-0.000000,-0.000000,nan,nan,nan,nan,nan,nan,nan
8,-0.000000,-0.000000,nan,nan,nan,nan,nan,nan,nan,nan
